In [6]:
import ROOT
# ROOT.gSystem.Load('liblorenzetti')
from lzt_utils.dataset import LztDataset
import mplhep
import matplotlib.pyplot as plt
from pathlib import Path
ROOT.EnableImplicitMT(2)
plt.style.use(mplhep.style.ROOT)

In [7]:
lzt_data = Path('/', 'root', 'ext_data', 'lorenzetti')
dataset_name = '2024_11_19_18_53_0000000000_jets'
dataset_path = lzt_data / dataset_name
output_dir = lzt_data / 'checks'
output_dir.mkdir(exist_ok=True, parents=True)
dataset = LztDataset.from_dir(dataset_path)
dataset

LztDataset(path=/root/ext_data/lorenzetti/2024_11_19_18_53_0000000000_jets, label=Jets et > 17GeV)

In [8]:
esd_rdf = dataset.get_esd_rdf(n_files=1)
esd_rdf.Describe()

Dataframe from TChain CollectionTree in file /root/ext_data/lorenzetti/2024_11_19_18_53_0000000000_jets/ESD/jf17.ESD.49.root

Property                Value
--------                -----
Columns in total           51
Columns from defines        0
Event loops run             0
Processing slots            2

Column                                          Type                                            Origin
------                                          ----                                            ------
CaloCellContainer_Cells                         ROOT::VecOps::RVec<xAOD::CaloCell_t>            Dataset
CaloCellContainer_Cells.descriptor_link         ROOT::VecOps::RVec<Int_t>                       Dataset
CaloCellContainer_Cells.deta                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.dphi                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.e                       ROOT::VecOps::RVec<Floa

In [9]:
esd_rdf.Define('CaloCluster_Clusters', "Lorenzetti::makeCaloCluster(EventSeedContainer_Seeds, CaloCellContainer_Cells)")
esd_rdf.Describe()

Dataframe from TChain CollectionTree in file /root/ext_data/lorenzetti/2024_11_19_18_53_0000000000_jets/ESD/jf17.ESD.49.root

Property                Value
--------                -----
Columns in total           51
Columns from defines        0
Event loops run             0
Processing slots            2

Column                                          Type                                            Origin
------                                          ----                                            ------
CaloCellContainer_Cells                         ROOT::VecOps::RVec<xAOD::CaloCell_t>            Dataset
CaloCellContainer_Cells.descriptor_link         ROOT::VecOps::RVec<Int_t>                       Dataset
CaloCellContainer_Cells.deta                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.dphi                    ROOT::VecOps::RVec<Float_t>                     Dataset
CaloCellContainer_Cells.e                       ROOT::VecOps::RVec<Floa

In [10]:
esd_rdf.Snapshot("CollectionTree", "esd_to_aod.root")

TypeError: Template method resolution failed:
  none of the 3 overloaded methods succeeded. Full details:
  ROOT::RDF::RResultPtr<ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void> > ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void>::Snapshot(experimental::basic_string_view<char,char_traits<char> > treename, experimental::basic_string_view<char,char_traits<char> > filename, initializer_list<string> columnList, const ROOT::RDF::RSnapshotOptions& options = ROOT::RDF::RSnapshotOptions()) =>
    TypeError: takes at least 3 arguments (2 given)
  ROOT::RDF::RResultPtr<ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void> > ROOT::RDF::RInterface<ROOT::Detail::RDF::RLoopManager,void>::Snapshot(experimental::basic_string_view<char,char_traits<char> > treename, experimental::basic_string_view<char,char_traits<char> > filename, const vector<string>& columnList, const ROOT::RDF::RSnapshotOptions& options = ROOT::RDF::RSnapshotOptions()) =>
    TypeError: takes at least 3 a

Error in <TTree::Branch>: The class requested (ROOT::VecOps::RVec<xAOD::EventInfo_t>) for the branch "EventInfoContainer_Events" is an instance of an stl collection and does not have a compiled CollectionProxy. Please generate the dictionary for this collection (ROOT::VecOps::RVec<xAOD::EventInfo_t>) to avoid to write corrupted data.
RDataFrame::Run: event loop was interrupted
Error in <TTree::Branch>: The class requested (ROOT::VecOps::RVec<xAOD::EventInfo_t>) for the branch "EventInfoContainer_Events" is an instance of an stl collection and does not have a compiled CollectionProxy. Please generate the dictionary for this collection (ROOT::VecOps::RVec<xAOD::EventInfo_t>) to avoid to write corrupted data.
RDataFrame::Run: event loop was interrupted
Error in <TTree::Branch>: The class requested (ROOT::VecOps::RVec<xAOD::EventInfo_t>) for the branch "EventInfoContainer_Events" is an instance of an stl collection and does not have a compiled CollectionProxy. Please generate the dictionar